<a href="https://colab.research.google.com/github/amandaliusa/cs155-kaggle/blob/master/catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_curve, auc

In [0]:
import tensorflow as tf
from tensorflow.python.data import Dataset
import keras
from keras.utils import to_categorical
from keras import models
from keras import layers

In [2]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv
Saving train.csv to train.csv


In [0]:
import io
df_train = pd.read_csv(io.BytesIO(uploaded['train.csv']))
df_test = pd.read_csv(io.BytesIO(uploaded['test.csv']))

In [0]:
# replace missing values with median (less sensitive to outliers)
df_train['opened_position_qty '].fillna(df_train['opened_position_qty '].median(),inplace=True)
df_test['opened_position_qty '].fillna(df_train['opened_position_qty '].median(),inplace=True)
df_train['closed_position_qty'].fillna(df_train['closed_position_qty'].median(),inplace=True)
df_test['closed_position_qty'].fillna(df_train['closed_position_qty'].median(), inplace=True)

In [0]:
df = pd.concat([df_train,df_test],sort=False)

In [0]:
# engineer new features by grouping like features 
bid_cols = ['bid1','bid2', 'bid3', 'bid4', 'bid5']
bid_vol_cols = ['bid1vol', 'bid2vol', 'bid3vol', 'bid4vol', 'bid5vol']
ask_cols = ['ask1', 'ask2', 'ask3', 'ask4', 'ask5',]
ask_vol_cols = ['ask1vol','ask2vol', 'ask3vol', 'ask4vol', 'ask5vol']
group_cols = {"bid_cols":bid_cols,"bid_vol_cols":bid_vol_cols,"ask_cols":ask_cols,"ask_vol_cols":ask_vol_cols}

In [7]:
for group in group_cols.keys():
    print(group)
    df[f"{group}_max"] = df[group_cols[group]].max(axis=1)
    df[f"{group}_min"] = df[group_cols[group]].min(axis=1)
    df[f"{group}_spread"] = df[f"{group}_max"].div(df[f"{group}_min"])
    #df[f"{group}_logsumexp"] = df[group_cols[group]].apply(logsumexp)
    
    df[f"{group}_max"] = df[group_cols[group]].max(axis=1)
    
df["last_price_div__mid"] = df["last_price"].div(df["mid"])

bid_cols
bid_vol_cols
ask_cols
ask_vol_cols


In [0]:
x_train = df.loc[~df.y.isna()]
y_train = x_train['y']
x_train = x_train.drop(['y'], axis=1)
x_test = df.loc[df.y.isna()]
x_test = x_test.drop(['y'], axis=1)

In [0]:
# Normalize data
from sklearn import preprocessing

x_train = df_train[df_train.columns[1:27]]
y_train = df_train['y']
x_test = df_test[df_test.columns[1:]]

# Normalize training data by subtracting mean and scaling to unit variance
std_scale = preprocessing.StandardScaler().fit(x_train)
x_train_norm = std_scale.transform(x_train)
x_train = pd.DataFrame(x_train_norm, index=x_train.index, columns=x_train.columns)

# Normalize testing data by using mean and SD of training set
x_test_norm = std_scale.transform(x_test)
x_test = pd.DataFrame(x_test_norm, index=x_test.index, columns=x_test.columns) 

In [0]:
# Split training and validation data 
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2)

In [10]:
pip install catboost

     |████████████████████████████████| 64.0MB 44kB/s 


In [11]:
from catboost import CatBoostClassifier
model = CatBoostClassifier(
    iterations=10000,
    learning_rate=0.1,
    task_type='GPU'
)
  
model.fit(
      x_train, y_train,
      eval_set=(x_val, y_val),
      verbose=10,
)  

print(model.get_best_score())


0:	learn: 0.6787772	test: 0.6789141	best: 0.6789141 (0)	total: 20.4ms	remaining: 3m 23s
10:	learn: 0.6247536	test: 0.6255340	best: 0.6255340 (10)	total: 209ms	remaining: 3m 9s
20:	learn: 0.6162779	test: 0.6173229	best: 0.6173229 (20)	total: 389ms	remaining: 3m 5s
30:	learn: 0.6137819	test: 0.6150430	best: 0.6150430 (30)	total: 531ms	remaining: 2m 50s
40:	learn: 0.6126258	test: 0.6140553	best: 0.6140553 (40)	total: 655ms	remaining: 2m 39s
50:	learn: 0.6119716	test: 0.6136149	best: 0.6136149 (50)	total: 777ms	remaining: 2m 31s
60:	learn: 0.6114804	test: 0.6132551	best: 0.6132551 (60)	total: 901ms	remaining: 2m 26s
70:	learn: 0.6110488	test: 0.6130064	best: 0.6130064 (70)	total: 1.02s	remaining: 2m 22s
80:	learn: 0.6107070	test: 0.6128327	best: 0.6128327 (80)	total: 1.15s	remaining: 2m 21s
90:	learn: 0.6103854	test: 0.6126657	best: 0.6126657 (90)	total: 1.28s	remaining: 2m 19s
100:	learn: 0.6101162	test: 0.6125725	best: 0.6125725 (100)	total: 1.41s	remaining: 2m 18s
110:	learn: 0.6098832	

In [12]:
y_pred = model.predict(x_val)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_val, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print('val auc: {}'.format(roc_auc))

val auc: 0.5859243689977026


In [14]:
from google.colab import drive
drive.mount('/content/gdrive')
#using model.save_weights() to save the weights of the model in HDF5 format
df_test['Predicted'] = model.predict_proba(x_test)[:,1]
df_test[["id","Predicted"]].to_csv('/content/gdrive/My Drive/submission_cat.csv', index=False)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
df_test['Predicted'] = model.predict_proba(x_test)[:,1]
df_test[["id","Predicted"]].to_csv('/content/gdrive/My Drive/submission_cat2.csv', index=False)

In [17]:
from sklearn.ensemble import RandomForestClassifier
model4 = RandomForestClassifier(n_estimators=64)
model4.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=64,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
from sklearn.ensemble import AdaBoostClassifier

boosting3 = AdaBoostClassifier(base_estimator=model4, n_estimators=3)   
boosting3.fit(x_train, y_train)
print(boosting3.score(x_train, y_train))
print(boosting3.score(x_val, y_val))

In [24]:
y_pred = boosting3.predict(x_val)
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_val, y_pred)
roc_auc = auc(false_positive_rate, true_positive_rate)
print('val auc: {}'.format(roc_auc))

val auc: 0.5735747605739127


In [0]:
df_test['Predicted'] = boosting3.predict_proba(x_test)[:,1]
df_test[["id","Predicted"]].to_csv('/content/gdrive/My Drive/submission_cat3.csv', index=False)